In [2]:
import pandas as pd
from pandas import json_normalize
from utils import read_json
import socceraction.vaep as vaep
import numpy as np
import pandas as pd
import json
import warnings
import os

import networkx as nx

from processing.eventing import StatsBombBasicPassingNetwork, StatsBombValuePassingNetwork

pd.set_option("display.max_columns", None)

warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# match_id = 68352
# match_id = 69225
match_id = 69235

lineups = read_json(f"data/eventing/lineups/{match_id}.json")

# team_name = lineups[0]["team_name"]
# team_name = "Real Madrid"
team_name = "Barcelona"

In [3]:
names_dict = {
    player["player_name"]: player["player_nickname"]
    for team in lineups
    for player in team["lineup"]
}
names_dict

{'Gnégnéri Yaya Touré': 'Yaya Touré',
 'Daniel Alves da Silva': 'Dani Alves',
 'Bojan Krkíc Pérez': 'Bojan',
 'Sergio Busquets i Burgos': 'Sergio Busquets',
 'Gerard Piqué Bernabéu': 'Gerard Piqué',
 'Andrés Iniesta Luján': 'Andrés Iniesta',
 'Lionel Andrés Messi Cuccittini': 'Lionel Messi',
 'Rafael Márquez Álvarez': 'Rafael Márquez',
 'Víctor Sánchez Mata': 'Víctor Sánchez',
 'Thierry Henry': None,
 "Samuel Eto''o Fils": "Samuel Eto'o",
 'Carles Puyol i Saforcada': 'Carles Puyol',
 'Xavier Hernández Creus': 'Xavi',
 'Víctor Valdés Arribas': 'Víctor Valdés',
 'Jesús Gámez Duarte': 'Jesús Gámez',
 'Adrián López Álvarez': 'Adrián López',
 'Eliseu Pereira dos Santos': 'Eliseu',
 'Albert Luque Martos': 'Albert Luque',
 'Sérgio Paulo Barbosa Valente': 'Duda',
 'Miguel Ángel Lozano Ayala': 'Miguel Ángel',
 'Ignacio Pérez Santamaría': 'Nacho',
 'Hélder Miguel Do Rosario Data': 'Hélder Rosario',
 'Antonio Galdeano Benítez': 'Apoño',
 'Fernando Miguel Fernández Escribano': 'Fernando Fernández'

In [4]:
events = read_json(f"data/eventing/events/{match_id}.json")
df_events = json_normalize(events, sep="_").assign(match_id=match_id)

In [5]:
# We select all successful passes done by the selected team before the minute
# of the first substitution or red card.

first_red_card_minute = df_events[df_events.foul_committed_card_name.isin(["Second Yellow", "Red Card"])].minute.min()
first_substitution_minute = df_events[df_events.type_name == "Substitution"].minute.min()
max_minute = df_events.minute.max()

num_minutes = min(first_substitution_minute, first_red_card_minute, max_minute)

- pass_end_location -> avg distance from opp goal
- pass_type_name -> no of each
- pass_through_ball -> %
- pass completion rate
- avg no of passes
- no of passes received by most advanced player
- avg position distance (exclude goalkeeper)
- Average Number of Unique Passing Links per Player
- Betti Number 0

In [6]:
# Taking into account all legitimate passes. Don't care about the same playing eleven on the field.

df_passes = df_events[
    (df_events.type_name == "Pass")
    & (df_events.pass_outcome_name != "Injury Clearance")
    & (df_events.team_name == team_name)
    # & (df_events.minute < num_minutes)
].copy()

In [7]:
df_passes.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,off_camera,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,carry_end_location,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,ball_receipt_outcome_name,under_pressure,duel_type_id,duel_type_name,clearance_aerial_won,clearance_head,clearance_body_part_id,clearance_body_part_name,ball_recovery_recovery_failure,foul_committed_type_id,foul_committed_type_name,counterpress,duel_outcome_id,duel_outcome_name,clearance_right_foot,pass_cross,out,interception_outcome_id,interception_outcome_name,dribble_outcome_id,dribble_outcome_name,foul_committed_card_id,foul_committed_card_name,shot_statsbomb_xg,shot_end_location,shot_outcome_id,shot_outcome_name,shot_technique_id,shot_technique_name,shot_body_part_id,shot_body_part_name,shot_type_id,shot_type_name,shot_freeze_frame,goalkeeper_position_id,goalkeeper_position_name,goalkeeper_technique_id,goalkeeper_technique_name,goalkeeper_type_id,goalkeeper_type_name,goalkeeper_outcome_id,goalkeeper_outcome_name,clearance_left_foot,foul_won_defensive,foul_won_advantage,pass_switch,pass_aerial_won,pass_assisted_shot_id,pass_shot_assist,shot_key_pass_id,shot_first_time,goalkeeper_end_location,goalkeeper_body_part_id,goalkeeper_body_part_name,dribble_overrun,dribble_nutmeg,foul_committed_advantage,block_offensive,pass_inswinging,pass_technique_id,pass_technique_name,shot_aerial_won,miscontrol_aerial_won,pass_cut_back,substitution_outcome_id,substitution_outcome_name,substitution_replacement_id,substitution_replacement_name,pass_goal_assist,block_deflection,dribble_no_touch,50_50_outcome_id,50_50_outcome_name,match_id
4,4352db01-f99f-4a83-93dc-c5ffb8f131a7,5,1,00:00:00.818,0,0,2,0.1600,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,[5117349c-88f6-48a1-837d-99c16d0bb5b1],True,"[61.0, 41.0]",5503.0,Lionel Andrés Messi Cuccittini,17.0,Right Wing,19298.0,Samuel Eto''o Fils,0.900000,-1.570796,1.0,Ground Pass,"[61.0, 40.1]",40.0,Right Foot,65.0,Kick Off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69235
6,88ca9438-de87-4cf0-a1bf-74c2aac14ca7,7,1,00:00:00.978,0,0,2,1.1590,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,[2b1f5f12-db82-4eb7-b17b-2b8bdd8631b7],NaN,"[60.9, 40.0]",19298.0,Samuel Eto''o Fils,23.0,Center Forward,3087.0,Gnégnéri Yaya Touré,7.368175,-3.005454,1.0,Ground Pass,"[53.6, 39.0]",40.0,Right Foot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69235
9,5eef11c9-1fa4-4bdf-bfe1-f142cd742260,10,1,00:00:03.765,0,3,2,3.4359,30,Pass,217,Barcelona,9,From Kick Off,217,Barcelona,NaN,NaN,[c00a1e47-03cd-4236-aaef-caf87e6b3ea2],NaN,"[53.2, 44.5]",3087.0,Gnégnéri Yaya Touré,10.0,Center Defensive Midfield,5213.0,Gerard Piqué Bernabéu,27.392334,-2.748206,1.0,Ground Pass,"[27.9, 34.0]",38.0,Left Foot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69235
12,f7237d39-8c74-4b9b-90b7-393f766f4330,1

In [8]:
# Function to convert location data to points
def _statsbomb_to_point(location, max_width=120, max_height=80):
    """
    Convert a point's coordinates from a StatsBomb's range to 0-1 range.
    """
    return location[0] / max_width, 1 - (location[1] / max_height)

# Euclidean distance function
def _calculate_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

In [9]:
# avg_dist_opp_goal -> avg distance from opp goal - pass_end_location

def calculate_average_distance_from_goal(df, fixed_point):
    """
    Calculate the average distance from a fixed point in the DataFrame.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the 'location' column.
    - fixed_point (tuple): A tuple (x, y) representing the fixed point.

    Returns:
    - float: The average distance from the fixed point to all points in the 'location' column.
    """

    # Apply transformations and calculate distances
    if 'location' in df.columns:
        distances = df["location"].apply(
            lambda x: _calculate_distance(fixed_point, _statsbomb_to_point(x))
        )

        return distances.mean()
    
    return 0

goal = (1, 0.5)
average_distance = calculate_average_distance_from_goal(df_passes, goal)
print("Average Distance:", average_distance)

Average Distance: 0.6066217110955285


In [10]:
# pass_type_name -> no of each

def count_pass_type(df, dropna=True, normalize=False):
    """
    Count the occurrences of each unique pass type in a DataFrame.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.

    Returns:
    - pd.Series: A Series with the count of each unique pass type.
    """
    if 'pass_type_name' in df.columns:
        pass_type_counts = df.pass_type_name.value_counts(normalize=normalize, dropna=dropna)
        return pass_type_counts

    return pd.Series(
        {
            "Recovery": 0,
            "Throw-in": 0,
            "Free Kick": 0,
            "Goal Kick": 0,
            "Interception": 0,
            "Corner": 0,
            "Kick Off": 0,
        }
    )

In [11]:
count_pass_type(df_passes)

pass_type_name
Recovery        78
Throw-in        44
Free Kick       16
Goal Kick        8
Interception     6
Corner           4
Kick Off         2
Name: count, dtype: int64

In [12]:
def calculate_through_ball_percentage(df, dropna=True):
    """
    Calculate the percentage of through balls in a given DataFrame column.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.
    - column_name (str): Name of the column indicating whether a pass is a through ball.

    Returns:
    - float: The percentage of through balls relative to all entries in the column, excluding NA values.
    """

    if 'pass_through_ball' in df.columns:

        # Calculate the number of through balls
        through_balls_count = df.pass_through_ball.sum()  # True values are counted as 1, False as 0

        if dropna:
            # Calculate the total number of non-NA entries
            total_passes = df.pass_through_ball.count()
        else:
            # Calculate the total number of entries
            total_passes = df.pass_through_ball.shape[0]

        # Calculate the percentage of through balls
        if total_passes > 0:
            through_ball_percentage = (through_balls_count / total_passes) * 100
            return through_ball_percentage
        else:
            return 0
        
    else:
        return 0

through_ball_percentage = calculate_through_ball_percentage(df_passes)
print("Percentage of Through Balls:", through_ball_percentage)

Percentage of Through Balls: 0


In [13]:
# pass completion rate

def calculate_pass_completion_rate(df):
    """
    Calculate the pass completion rate from the pass outcome column.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.
    - outcome_column (str): Name of the column indicating the outcome of a pass.

    Returns:
    - float: The pass completion rate as a percentage.
    """
    if 'pass_outcome_name' in df.columns:
        # Count the number of successful passes (NaN in the outcome column)
        successful_passes_count = df.pass_outcome_name.isna().sum()
        # Count the total number of passes
        total_passes = len(df.pass_outcome_name) 

        # Calculate the pass completion rate
        if total_passes > 0:
            completion_rate = (successful_passes_count / total_passes) * 100
            return completion_rate
        else:
            return 0
    else:
        return 0


pass_completion_rate = calculate_pass_completion_rate(df_passes)
print("Pass Completion Rate:", pass_completion_rate)

Pass Completion Rate: 53.98936170212766


In [14]:
# no of passes received by most advanced player

def calculate_avg_passes_per_player(df):
    """
    Calculate the average number of passes per player.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.

    Returns:
    - float: Average number of passes per player.
    """
    
    # Total number of passes
    total_passes = df.shape[0]
    # Number of unique players
    num_unique_players = df.player_id.nunique()

    # Calculate the average number of passes per player
    if num_unique_players > 0:
        avg_passes_per_player = total_passes / num_unique_players
        return avg_passes_per_player
    else:
        return 0 

avg_passes_per_player = calculate_avg_passes_per_player(df_passes)
print("Average Passes Per Player:", avg_passes_per_player)

Average Passes Per Player: 26.857142857142858


In [15]:
# number of passes received by the most advanced player

def calculate_passes_to_most_advanced_player(df):
    """
    Calculate the number of passes received by the most advanced player based on the median of their pass origin positions.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.

    Returns:
    - int: Number of passes received by the most advanced player.
    """

    # Calculate each player's average pass origin's coordinates
    df["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
    df["origin_pos_y"] = df.location.apply(lambda x: _statsbomb_to_point(x)[1])

    # Determine the median position for each player
    player_position = df.groupby("player_name").agg({"origin_pos_x": "median", "origin_pos_y": "median"})

    players_distances = player_position.apply(
        lambda x: _calculate_distance((0, 0.5), (x["origin_pos_x"], x["origin_pos_y"])), axis=1)

    # Identify the most advanced player
    most_advanced_player = players_distances.idxmax()
    # Count the number of passes received by this player
    passes_received = df[df["pass_recipient_name"] == most_advanced_player].shape[0]

    return most_advanced_player, passes_received

# Example usage:
# Assuming df_passes is your DataFrame and it includes necessary columns

most_advanced_player, number_of_passes_received = (
    calculate_passes_to_most_advanced_player(df_passes)
)
print("Most Advanced Player:", most_advanced_player)
print("Number of Passes Received by the Most Advanced Player:", number_of_passes_received)

Most Advanced Player: Bojan Krkíc Pérez
Number of Passes Received by the Most Advanced Player: 9


In [16]:
# avg position distance (exclude goalkeeper)

def calculate_avg_x_position_excluding_goalkeepers(df):
    """
    Calculate the average x position for all players excluding goalkeepers.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the player data with positions.
    - position_column (str): Column name where player roles or positions are defined.
    - x_position_column (str): Column name where x positions are stored.

    Returns:
    - float: Average x position for non-goalkeepers.
    """

    # Filter out goalkeepers
    non_goalkeepers = df[df["position_name"] != "Goalkeeper"]

    # Calculate the average x position for non-goalkeepers
    non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])
    average_x_position = non_goalkeepers["origin_pos_x"].mean()

    return average_x_position


avg_x_position_non_gk = calculate_avg_x_position_excluding_goalkeepers(df_passes)
print("Average X Position (excluding goalkeepers):", avg_x_position_non_gk)

Average X Position (excluding goalkeepers): 0.520929307116105


/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_40588/1849874024.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_goalkeepers["origin_pos_x"] = df.location.apply(lambda x: _statsbomb_to_point(x)[0])


In [17]:
# Average Number of Unique Passing Links per Player

def average_unique_passing_links(df):
    """
    Calculate the average number of unique passing links per player.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.
    
    Returns:
    - float: Average number of unique passing links per player.
    """

    # Group by player and list unique recipients
    unique_links_per_player = df.groupby("player_name")["pass_recipient_name"]\
        .agg(lambda x: x.dropna().nunique())

    # Calculate the average number of unique links
    average_links = unique_links_per_player.mean()

    return average_links

avg_links = average_unique_passing_links(df_passes)
print("Average Number of Unique Passing Links per Player:", avg_links)

Average Number of Unique Passing Links per Player: 7.642857142857143


In [18]:
def get_name_or_default(name):
    if not name or pd.isna(name):  # Checks if name is None or NaN
        return None
    # Safely gets the value from the dictionary or returns the name itself
    return names_dict.get(name, name) if names_dict.get(name) is not None else name


# If available, use player's nickname instead of full name to optimize space in plot
# df_new["pass_recipient_name"] = df_new.pass_recipient_name.apply(lambda x: None if not x else names_dict[x] if names_dict[x] else x)
df_passes["pass_recipient_name"] = df_passes["pass_recipient_name"].apply(
    get_name_or_default
)
df_passes["player_name"] = df_passes.player_name.apply(
    lambda x: names_dict[x] if names_dict[x] else x
)

In [19]:
predictions_h5 = os.path.join("data/eventing", "predictions.h5")

spadl_h5 = os.path.join("data/eventing", "spadl-statsbomb.h5")

actiontypes = pd.read_hdf(spadl_h5, "actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")
results = pd.read_hdf(spadl_h5, "results")
players = pd.read_hdf(spadl_h5, "players")
teams = pd.read_hdf(spadl_h5, "teams")
actions = pd.read_hdf(spadl_h5, "actions/game_{0}".format(match_id))

actions = (
    actions.merge(actiontypes, how="left")
    .merge(results, how="left")
    .merge(bodyparts, how="left")
    .merge(players, how="left")
    .merge(teams, how="left")
)

In [20]:
preds = pd.read_hdf(predictions_h5, "game_{0}".format(match_id))
values = vaep.value(actions, preds.scores, preds.concedes)

df_vaep = pd.concat([actions, preds, values], axis=1)
# df_vaep["player_name"] = df_vaep.apply(lambda x: x["player_nickname"] if x["player_nickname"] else x["player_name"], axis=1)
df_vaep["player_name"] = df_vaep.player_name.apply(lambda x: names_dict[x] if names_dict[x] else x)

/Users/ananshar/anaconda3/envs/soccer_analysis/lib/python3.10/site-packages/socceraction/vaep.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0465' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  prev_scores[corner_idx] = 0.046500


In [21]:
def calculate_team_pass_value(df_passes, df_vaep):
    # Merge df_passes and df_vaep on timestamp and player_name
    df_result = pd.merge(df_passes[["timestamp", "player_name", "pass_recipient_name"]], df_vaep,
        on=["timestamp", "player_name"], how="left")

    # Filter out negative actions
    df_result["vaep_value"] = df_result.vaep_value.apply(lambda x: x if x >= 0 else None)

    # Group by player_name and calculate mean vaep_value
    player_pass_value = df_result.groupby("player_name").agg(pass_value=("vaep_value", "mean"))

    return player_pass_value.mean().values[0]

team_pass_value = calculate_team_pass_value(df_passes, df_vaep)
print("Average Team VAEP Value:", team_pass_value)

Average Team VAEP Value: 0.01752521122894877


In [22]:
df_passes_no_gk = df_passes[df_passes["position_name"] != "Goalkeeper"]
player_position = df_passes.groupby("player_name").agg({"origin_pos_x": "median", "origin_pos_y": "median"})
player_position_no_gk = df_passes_no_gk.groupby("player_name").agg({"origin_pos_x": "median", "origin_pos_y": "median"})


def calculate_avg_pass_distance(df):
    # Calculate the Euclidean distance between origin and end positions for each pass
    df["origin_pos"] = df.location.apply(lambda x: np.array(_statsbomb_to_point(x)))
    df["end_pos"] = df.pass_end_location.apply(lambda x: np.array(_statsbomb_to_point(x)))


    df["pass_distance"] = df.apply(
        lambda row: np.linalg.norm(row["end_pos"] - row["origin_pos"]), axis=1
    )

    # Compute the average pass distance
    return df["pass_distance"].mean()


# Example usage with the adjusted coordinate transformation
average_pass_distance = calculate_avg_pass_distance(df_passes)
print("Average Pass Distance:", average_pass_distance)

Average Pass Distance: 0.21809361230574542


In [23]:
def calculate_backline_distance(df, player_position):
    # Find the four players whose median x-coordinates are closest to zero

    df["origin_pos_x"] = df.location.apply(lambda x: np.array(_statsbomb_to_point(x)[0]))

    four_closest_to_goal = player_position.nsmallest(4, "origin_pos_x")
    # Calculate the average Euclidean distance of these four players from the point [0, 0.5] on the field
    goal_point = np.array([0, 0.5])
    four_closest_to_goal["distance_to_goal"] = four_closest_to_goal.apply(
        lambda row: np.linalg.norm(
            np.array([row["origin_pos_x"], row["origin_pos_y"]]) - goal_point
        ),
        axis=1,
    )

    # Return the average of these distances
    return four_closest_to_goal["distance_to_goal"].mean()

# Calculate the average backline distance using the function
avg_backline_distance = calculate_backline_distance(df_passes, player_position_no_gk)
print("Average Distance of the Back Four from Own Goal:", avg_backline_distance)

Average Distance of the Back Four from Own Goal: 0.4208461911951512


In [24]:
def calculate_front_three_distance(df, player_position):

    df["origin_pos_x"] = df.location.apply(
        lambda x: np.array(_statsbomb_to_point(x)[0])
    )

    # Find the three players whose median x-coordinates are the largest (furthest forward)
    three_furthest_forward = player_position.nlargest(3, "origin_pos_x")

    # Calculate the average distance of these three players from their own goal line (x = 0)
    average_distance_to_goal = three_furthest_forward["origin_pos_x"].mean()

    # Return the average distance to their own goal
    return average_distance_to_goal

# Calculate the average front three distance using the function
avg_front_three_distance = calculate_front_three_distance(df_passes, player_position_no_gk)
print("Average Distance of the Front Three from Own Goal:", avg_front_three_distance)

Average Distance of the Front Three from Own Goal: 0.6890277777777779


In [25]:
def calculate_avg_time_between_passes(df):
    team_passes = df.copy()
    # Ensure the DataFrame is sorted by timestamp within each possession
    team_passes = team_passes.sort_values(by=["possession", "timestamp"])
    # Convert timestamps to pandas datetime objects if they are not already
    team_passes["timestamp"] = pd.to_datetime(team_passes["timestamp"], errors="coerce")
    # Check for any conversion failures
    if team_passes["timestamp"].isna().any():
        print(
            "Warning: Some timestamps couldn't be converted and will be treated as NaN in calculations."
        )
    # Calculate the time difference between consecutive passes within the same possession
    team_passes["time_diff"] = (
        team_passes.groupby("possession")["timestamp"].diff().dt.total_seconds()
    )
    # Exclude the first pass in each possession as it does not have a preceding pass to calculate a time difference
    team_passes = team_passes.dropna(subset=["time_diff"])
    # Group by possession ID and calculate the mean time difference between passes for each possession
    mean_time_per_possession = team_passes.groupby("possession")["time_diff"].mean()
    # Calculate the global mean of these possession averages
    global_mean_time_between_passes = mean_time_per_possession.mean()

    return global_mean_time_between_passes


# Assuming df_new is your DataFrame, and "Barcelona" is the team you're analyzing
average_time_between_passes = calculate_avg_time_between_passes(df_passes)
print(f"Average Time Between Passes for {team_name}:", average_time_between_passes)

Average Time Between Passes for Barcelona: 3.734752516453736


/var/folders/gl/s_1pz2z130g9ct6hvptjf5fh0000gn/T/ipykernel_40588/2725290664.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  team_passes["timestamp"] = pd.to_datetime(team_passes["timestamp"], errors="coerce")


In [26]:
def calculate_global_avg_passes_per_possession(df):
    # Filter passes by team name
    team_passes = df.copy()

    # Group by possession ID and count the number of passes in each possession
    passes_per_possession = team_passes.groupby("possession").size()

    # Calculate the global average number of passes per possession
    global_avg_passes = passes_per_possession.mean()

    return global_avg_passes


# Assuming df_new is your DataFrame, and "Barcelona" is the team you're analyzing
average_passes_per_possession = calculate_global_avg_passes_per_possession(df_passes)
print(
    f"Average Number of Passes Per Possession for {team_name}:",
    average_passes_per_possession,)

Average Number of Passes Per Possession for Barcelona: 2.647887323943662


In [27]:
def calculate_play_pattern_count(df, dropna=True, normalize=False):
    """
    Count the occurrences of each unique pass type in a DataFrame.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the pass data.

    Returns:
    - pd.Series: A Series with the count of each unique pass type.
    """

    if "play_pattern_name" in df.columns:
        play_pattern_type_counts = df.play_pattern_name.value_counts(
            normalize=normalize, dropna=dropna
        )

        return play_pattern_type_counts

    return pd.Series(
        {
            "Regular Play": 0,
            "From Throw In": 0,
            "From Free Kick": 0,
            "From Keeper": 0,
            "From Goal Kick": 0,
            "From Corner": 0,
            "From Kick Off": 0
        }
    )

play_pattern_count = calculate_play_pattern_count(df_passes, normalize=True)

In [28]:
from_recovery_passes = play_pattern_count.get("Regular Play", 0)
from_throw_in_passes = play_pattern_count.get("From Throw In", 0)
from_free_kick_passes = play_pattern_count.get("From Free Kick", 0)
from_corner_passes = play_pattern_count.get("From Corner", 0)
from_goal_kick_passes = play_pattern_count.get("From Goal Kick", 0)
from_interception_passes = play_pattern_count.get("From Keeper", 0)
from_kick_off_passes = play_pattern_count.get("From Kick Off", 0)

In [29]:
print("From Recovery Passes:", from_recovery_passes)
print("From Throw-in Passes:", from_throw_in_passes)
print("From Free Kick Passes:", from_free_kick_passes)
print("From Corner Passes:", from_corner_passes)
print("From Goal Kick Passes:", from_goal_kick_passes)
print("From Interception Passes:", from_interception_passes)
print("From Kick Off Passes:", from_kick_off_passes)

From Recovery Passes: 0.375
From Throw-in Passes: 0.3723404255319149
From Free Kick Passes: 0.09574468085106383
From Corner Passes: 0.015957446808510637
From Goal Kick Passes: 0.0851063829787234
From Interception Passes: 0.010638297872340425
From Kick Off Passes: 0.023936170212765957


In [30]:
def calculate_directness_of_play(df):
    # Extract the x-coordinates from the normalized position arrays

    df["origin_pos_x"] = df.location.apply(lambda x: np.array(_statsbomb_to_point(x)[0]))
    df["end_pos_x"] = df.pass_end_location.apply(lambda x: np.array(_statsbomb_to_point(x)[0]))

    # df = df.copy()
    # Calculate the directness as the difference in x-coordinates
    df["directness"] = df["end_pos_x"] - df["origin_pos_x"]

    # Calculate the average directness of all passes
    average_directness = df["directness"].mean()

    return average_directness


# Assuming df2_new is your DataFrame
average_directness = calculate_directness_of_play(df_passes)
print("Average Directness of Play:", average_directness)

Average Directness of Play: 0.12459884751773048


In [36]:
df_passes["pair_key"] = df_passes.apply(lambda x: (
        "_".join(sorted([x["player_name"], x["pass_recipient_name"]]))
        if x["player_name"] is not None and x["pass_recipient_name"] is not None
        else None
    ), axis=1)

pair_pass_count = df_passes.groupby("pair_key").size().to_frame("num_passes")
# pair_pass_value = df_passes.groupby("pair_key").size().to_frame("pass_value")
pair_pass_count

,num_passes
pair_key,
Andrés Iniesta_Carles Puyol,8
Andrés Iniesta_Dani Alves,1
Andrés Iniesta_Gerard Piqué,5
Andrés Iniesta_Lionel Messi,4
Andrés Iniesta_Samuel Eto'o,6
...,...
Thierry Henry_Víctor Valdés,1
Thierry Henry_Xavi,5
Thierry Henry_Yaya Touré,2


In [37]:
pair_pass_count = pair_pass_count.reset_index()
pair_pass_count

,pair_key,num_passes
0,Andrés Iniesta_Carles Puyol,8
1,Andrés Iniesta_Dani Alves,1
2,Andrés Iniesta_Gerard Piqué,5
3,Andrés Iniesta_Lionel Messi,4
4,Andrés Iniesta_Samuel Eto'o,6
...,...,...
65,Thierry Henry_Víctor Valdés,1
66,Thierry Henry_Xavi,5
67,Thierry Henry_Yaya Touré,2
68,Víctor Valdés_Xavi,2


In [32]:
pair_pass_count["pair_key"] = pair_pass_count.index
pair_pass_count = pair_pass_count.reset_index(drop=True)
pair_pass_count

,num_passes,pair_key
0,8,Andrés Iniesta_Carles Puyol
1,1,Andrés Iniesta_Dani Alves
2,5,Andrés Iniesta_Gerard Piqué
3,4,Andrés Iniesta_Lionel Messi
4,6,Andrés Iniesta_Samuel Eto'o
...,...,...
65,1,Thierry Henry_Víctor Valdés
66,5,Thierry Henry_Xavi
67,2,Thierry Henry_Yaya Touré
68,2,Víctor Valdés_Xavi


In [33]:
df = pd.DataFrame(pair_pass_count)
df.set_index("pair_key", inplace=True)

# Function to calculate Betti numbers
def calculate_betti_numbers(graph):
    # Compute the number of connected components (Betti 0)
    betti_0 = nx.number_connected_components(graph)

    # Compute the number of edges and nodes
    num_edges = graph.number_of_edges()
    num_nodes = graph.number_of_nodes()

    # Compute the rank of the cycle space (Betti 1)
    betti_1 = num_edges - num_nodes + betti_0

    return betti_0, betti_1

# Create a graph from the DataFrame
G = nx.Graph()

# Add edges to the graph
for pair_key, num_passes in df["num_passes"].items():
    player1, player2 = pair_key.split("_")
    G.add_edge(player1, player2, weight=num_passes)

# Calculate Betti numbers for the graph
betti_0, betti_1 = calculate_betti_numbers(G)
print(f"Betti 0: {betti_0}")
print(f"Betti 1: {betti_1}")

# Plot the graph for visualization
pos = nx.spring_layout(G)
# plt.figure(figsize=(10, 7))
# nx.draw(
#     G,
#     pos,
#     with_labels=True,
#     node_color="lightblue",
#     edge_color="gray",
#     node_size=3000,
#     font_size=10,
#     font_weight="bold",
# )
# labels = nx.get_edge_attributes(G, "weight")
# nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
# plt.title("Pass Network Graph")
# plt.show()

Betti 0: 1
Betti 1: 57


In [34]:
season_data = "data/eventing/matches/11/41.json"

In [35]:

season

NameError: name 'season' is not defined

In [ ]:
df_season = json_normalize(season, sep="_")
df_season.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_competition_id,competition_country_name,competition_competition_name,season_season_id,season_season_name,home_team_home_team_id,home_team_home_team_name,home_team_home_team_gender,home_team_home_team_group,home_team_country_id,home_team_country_name,home_team_managers,away_team_away_team_id,away_team_away_team_name,away_team_away_team_gender,away_team_away_team_group,away_team_country_id,away_team_country_name,away_team_managers,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,69225,2008-12-13,22:00:00.000,2,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,15,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",220,Real Madrid,male,None,214,Spain,"[{'id': 3899, 'name': 'Juan de la Cruz Ramos C...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,1011,Luis Medina Cantalejo,214,Spain
1,69212,2008-10-04,22:00:00.000,6,1,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,6,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",212,Atlético Madrid,male,None,214,Spain,"[{'id': 3196, 'name': 'Javier Aguirre Onaindía...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,999,Eduardo Iturralde Gonzalez,214,Spain
2,69235,2008-11-01,20:00:00.000,1,4,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,9,11,Spain,La Liga,41,2008/2009,223,Málaga,male,None,214,Spain,"[{'id': 4989, 'name': 'Antonio Tapia Flores', ...",217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",1.1.0,2,2,1,Regular Season,346,Estadio La Rosaleda,214,Spain,2535,Carlos Velasco Carballo,214,Spain
3,69232,2008-10-25,22:00:00.000,5,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,8,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,NaN,403,Almería,male,None,214,Spain,NaN,1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,1007,Alfonso Perez Burrull,214,Spain
4,69216,2009-03-07,20:00:00.000,2,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,26,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",215,Athletic Club,male,None,214,Spain,"[{'id': 497, 'name': 'Joaquín de Jesús Caparró...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,221,Alberto Undiano Mallenco,214,Spain


In [ ]:
df_season.columns

Index(['match_id', 'match_date', 'kick_off', 'home_score', 'away_score',
       'match_status', 'match_status_360', 'last_updated', 'last_updated_360',
       'match_week', 'competition_competition_id', 'competition_country_name',
       'competition_competition_name', 'season_season_id',
       'season_season_name', 'home_team_home_team_id',
       'home_team_home_team_name', 'home_team_home_team_gender',
       'home_team_home_team_group', 'home_team_country_id',
       'home_team_country_name', 'home_team_managers',
       'away_team_away_team_id', 'away_team_away_team_name',
       'away_team_away_team_gender', 'away_team_away_team_group',
       'away_team_country_id', 'away_team_country_name', 'away_team_managers',
       'metadata_data_version', 'metadata_shot_fidelity_version',
       'metadata_xy_fidelity_version', 'competition_stage_id',
       'competition_stage_name', 'stadium_id', 'stadium_name',
       'stadium_country_id', 'stadium_country_name', 'referee_id',
       're

In [ ]:
def get_outcome(x):
    if x['home_score'] > x['away_score']:
        return "team1_wins"
    elif x['home_score'] < x['away_score']:
        return "team2_wins"
    else:
        return "draw"

df_season["outcome"] = df_season.apply(get_outcome, axis=1)

In [ ]:
df_season.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_competition_id,competition_country_name,competition_competition_name,season_season_id,season_season_name,home_team_home_team_id,home_team_home_team_name,home_team_home_team_gender,home_team_home_team_group,home_team_country_id,home_team_country_name,home_team_managers,away_team_away_team_id,away_team_away_team_name,away_team_away_team_gender,away_team_away_team_group,away_team_country_id,away_team_country_name,away_team_managers,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name,outcome
0,69225,2008-12-13,22:00:00.000,2,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,15,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",220,Real Madrid,male,None,214,Spain,"[{'id': 3899, 'name': 'Juan de la Cruz Ramos C...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,1011,Luis Medina Cantalejo,214,Spain,team1_wins
1,69212,2008-10-04,22:00:00.000,6,1,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,6,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",212,Atlético Madrid,male,None,214,Spain,"[{'id': 3196, 'name': 'Javier Aguirre Onaindía...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,999,Eduardo Iturralde Gonzalez,214,Spain,team1_wins
2,69235,2008-11-01,20:00:00.000,1,4,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,9,11,Spain,La Liga,41,2008/2009,223,Málaga,male,None,214,Spain,"[{'id': 4989, 'name': 'Antonio Tapia Flores', ...",217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",1.1.0,2,2,1,Regular Season,346,Estadio La Rosaleda,214,Spain,2535,Carlos Velasco Carballo,214,Spain,team2_wins
3,69232,2008-10-25,22:00:00.000,5,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,8,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,NaN,403,Almería,male,None,214,Spain,NaN,1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,1007,Alfonso Perez Burrull,214,Spain,team1_wins
4,69216,2009-03-07,20:00:00.000,2,0,available,scheduled,2020-07-29T05:00,2021-06-13T16:17:31.694,26,11,Spain,La Liga,41,2008/2009,217,Barcelona,male,None,214,Spain,"[{'id': 36, 'name': 'Josep Guardiola i Sala', ...",215,Athletic Club,male,None,214,Spain,"[{'id': 497, 'name': 'Joaquín de Jesús Caparró...",1.1.0,2,2,1,Regular Season,342,Spotify Camp Nou,214,Spain,221,Alberto Undiano Mallenco,214,Spain,team1_wins


In [ ]:
match_data["outcome"]